In [1]:
MERCHAINDISE = 'DOTUSDT'
DAYS = 10

# format = YYYY-MM-DD
# START_DATE = '2022-4-1'
# END_DATE = '2022-4-6'
START_DATE = None
END_DATE =  None

DATE_FORMAT = "%Y-%m-%d"

In [3]:
from apps.models.candlestick import Candlestick
from apps.models.merchandise_rate import MerchandiseRate
from apps.models.zone import Zone
from apps.helpers.draw_chart import draw_candlestick_use_zones
from apps.services.ochl_dataframe import add_hour_column, add_return_column
import matplotlib.pyplot as plt

%matplotlib inline

merchandise_rate = MerchandiseRate()
merchandise_rate_id = merchandise_rate.find_by_slug(MERCHAINDISE)
candlestick = Candlestick(merchandise_rate_id, 'hour', 121, "DESC", start_date=START_DATE, end_date=END_DATE)

data_prices = candlestick.to_df()

data_prices = add_hour_column(data_prices)
data_prices = add_return_column(data_prices)

zones = Zone.calculate_sideway_zone(data_prices)

for zone in zones:
    print(zone.__dict__)

draw_candlestick_use_zones(data_prices, zones).show()

{'start': 8, 'end': 14, 'high': 7.715, 'low': 7.846025}
{'start': 42, 'end': 50, 'high': 7.12066, 'low': 7.3187999999999995}
{'start': 53, 'end': 62, 'high': 6.999568, 'low': 7.229839999999999}
{'start': 62, 'end': 68, 'high': 6.8023175, 'low': 6.97715}
{'start': 69, 'end': 74, 'high': 6.705, 'low': 6.8149999999999995}
{'start': 77, 'end': 82, 'high': 6.7715000000000005, 'low': 6.9585}
{'start': 90, 'end': 120, 'high': 6.5275, 'low': 6.88691828125}


In [ ]:
from apps.models.candlestick import Candlestick
from apps.models.merchandise_rate import MerchandiseRate
from apps.models.zone import Zone
from apps.helpers.draw_chart import draw_candlestick_use_zones
from apps.services.ochl_dataframe import add_hour_column, add_return_column, add_average_oc_column, add_rolling_average_oc_column
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

merchandise_rate = MerchandiseRate()
merchandise_rate_id = merchandise_rate.find_by_slug(MERCHAINDISE)
candlestick = Candlestick(merchandise_rate_id, 'hour', 200, "DESC", start_date=START_DATE, end_date=END_DATE)

data_prices = candlestick.to_df()

data_prices = add_hour_column(data_prices)
data_prices = add_return_column(data_prices)
data_prices = add_average_oc_column(data_prices)
data_prices = add_rolling_average_oc_column(data_prices)


zones = Zone.calculate_sideway_zone(data_prices, 0.02)

# print(zones)

draw_candlestick_use_zones(data_prices, zones).show()

In [ ]:
Average_Weight = 1.0
Min_Zone_Width = 3
Tolerance = 20
Show_N_Last_Zones = 2
Num_Bars = 500
Start = 0
End = 0
Low = 0
High = 0

avgZoneHeight = 0
tolerancePt = Tolerance*0.001

numBars = 30
hasBrokenRange = False
zoneStart = 29
zoneEnd = 0
zoneSize = 0
zonesCounter = 0
zones = []

rangeHigh = data_prices.iloc[numBars - 1].high + tolerancePt
rangeLow = data_prices.iloc[numBars - 1].low - tolerancePt

for i in range(numBars - 2,-1,-1):
      zoneSize = zoneStart - i
      zoneEnd = i
      if data_prices.iloc[i].open > rangeHigh or data_prices.iloc[i].open < rangeLow or data_prices.iloc[i].close > rangeHigh or data_prices.iloc[i].close < rangeLow:
            if zoneSize >= Min_Zone_Width:
                  avgZoneHeight += abs(rangeHigh - rangeLow)

                  zonesCounter = zonesCounter + 1

            hasBrokenRange = True
            zoneStart = i
            rangeHigh = data_prices.iloc[i].high + tolerancePt;
            rangeLow = data_prices.iloc[i].low - tolerancePt;
      else:
            hasBrokenRange = False

avgZoneHeight /= zonesCounter * Average_Weight

for i in range(numBars - 2,-1,-1):
      zoneSize = zoneStart - i
      zoneEnd = i
      if data_prices.iloc[i].open > rangeHigh or data_prices.iloc[i].open < rangeLow or data_prices.iloc[i].close > rangeHigh or data_prices.iloc[i].close < rangeLow:
            if zoneSize >= Min_Zone_Width:
                  zones.append({'start': zoneStart, 'end': zoneEnd + 1, 'low': rangeLow, 'high': rangeHigh})

                  zonesCounter = zonesCounter + 1

            hasBrokenRange = True
            zoneStart = i
            rangeHigh = data_prices.iloc[i].high + tolerancePt;
            rangeLow = data_prices.iloc[i].low - tolerancePt;

            if abs(data_prices.iloc[i].high - data_prices.iloc[i].low) >= avgZoneHeight:
                  zoneStart = -1
                  rangeHigh = 0
                  rangeLow = 0
                  continue

      else:
            hasBrokenRange = False

if hasBrokenRange == False and zoneSize >= Min_Zone_Width:
      zones.append({'start': zoneStart, 'end': 0, 'low': rangeLow, 'high': rangeHigh})

print(zones)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(data_prices['rolling_average_oc'], label='rolling_avg')
plt.figure(figsize=(6, 6))

plt.legend()

plt.show()

In [ ]:
data_prices_0_6 = data_prices[data_prices['hour'] < 7]

plt.figure(figsize=(20,10))
plt.plot(data_prices_0_6['rolling_average_oc'], label='rolling_avg')
plt.figure(figsize=(6, 6))

plt.legend()

plt.show()

In [ ]:
Average_Weight = 1.0
Min_Zone_Width = 3
Tolerance = 20
Show_N_Last_Zones = 2
Num_Bars = 500
Start = 0
End = 0
Low = 0
High = 0

avgZoneHeight = 0
tolerancePt = Tolerance*0.01

numBars = 30
hasBrokenRange = False
zoneStart = 29
zoneEnd = 0
zoneSize = 0
zonesCounter = 0
zones = []

rangeHigh = data_prices.iloc[numBars - 1].high + tolerancePt
rangeLow = data_prices.iloc[numBars - 1].low - tolerancePt

for i in range(numBars - 2,-1,-1):
      zoneSize = zoneStart - i
      zoneEnd = i
      if data_prices.iloc[i].open > rangeHigh or data_prices.iloc[i].open < rangeLow or data_prices.iloc[i].close > rangeHigh or data_prices.iloc[i].close < rangeLow:
            if zoneSize >= Min_Zone_Width:
                  avgZoneHeight += abs(rangeHigh - rangeLow)

                  zonesCounter = zonesCounter + 1

            hasBrokenRange = True
            zoneStart = i
            rangeHigh = data_prices.iloc[i].high + tolerancePt;
            rangeLow = data_prices.iloc[i].low - tolerancePt;
      else:
            hasBrokenRange = False

avgZoneHeight /= zonesCounter * Average_Weight

for i in range(numBars - 2,-1,-1):
      zoneSize = zoneStart - i
      zoneEnd = i
      if data_prices.iloc[i].open > rangeHigh or data_prices.iloc[i].open < rangeLow or data_prices.iloc[i].close > rangeHigh or data_prices.iloc[i].close < rangeLow:
            if zoneSize >= Min_Zone_Width:
                  zones.append({'start': zoneStart, 'end': zoneEnd + 1, 'low': rangeLow, 'high': rangeHigh})

                  zonesCounter = zonesCounter + 1

            hasBrokenRange = True
            zoneStart = i
            rangeHigh = data_prices.iloc[i].high + tolerancePt;
            rangeLow = data_prices.iloc[i].low - tolerancePt;

            if abs(data_prices.iloc[i].high - data_prices.iloc[i].low) >= avgZoneHeight:
                  zoneStart = -1
                  rangeHigh = 0
                  rangeLow = 0
                  continue

      else:
            hasBrokenRange = False

if hasBrokenRange == False and zoneSize >= Min_Zone_Width:
      zones.append({'start': zoneStart, 'end': 0, 'low': rangeLow, 'high': rangeHigh})

print(zones)


# data_prices.iloc[[28]]

# double rangeHigh = High[numBars - 1] + tolerancePt, rangeLow = Low[numBars - 1] - tolerancePt;
# int zoneStart = numBars - 1, zoneEnd = 0, zoneSize = 0, zonesCounter = 0;

# ArrayFree(zones);

# // Iterate through all bars, beginning at the very first bar of the chart
# for (int i = numBars - 2; i >= 0; i--)
# {
#    zoneSize = zoneStart - i;
#    zoneEnd = i;

#    // Check if a bar opened or closed above range high or range low
#    if (Open[i] > rangeHigh || Open[i] < rangeLow || Close[i] > rangeHigh || Close[i] < rangeLow)
#    {
#       // Check if number of bars in zone is bigger or equal the required width
#       if (zoneSize >= Min_Zone_Width)
#       {
#          // On first pass, add up the height of all zones found
#          if (pass == 1)
#          {
#             avgZoneHeight += MathAbs(rangeHigh - rangeLow);
#          }
      
#          // On second pass, add the zone to a list
#          if (pass == 2)
#             AddZone(zones, zoneStart, zoneEnd + 1, rangeLow, rangeHigh);
      
#          zonesCounter++;
#       }

#       hasBrokenRange = true;
#       zoneStart = i;
#       rangeHigh = High[i] + tolerancePt;
#       rangeLow = Low[i] - tolerancePt;

#       // Pass 2: If bar height is bigger than the expected average zone height, skip this bar
#       if (pass == 2 && MathAbs(High[i] - Low[i]) >= avgZoneHeight)
#       {
#          zoneStart = -1;
#          rangeHigh = 0;
#          rangeLow = 0;
#          continue;
#       }
#    }
#    else
#       hasBrokenRange = false;
# }

# // After first pass, calculate the average zone height
# // and weight it a bit more to filter out big movements
# if (pass == 1)
      #    avgZoneHeight /= zonesCounter * Average_Weight;